### 该文件主要用于功能测试实验
进行了以下几个实验：     
0）自定义CNN网络，自定义网络训练速度快，但是精度非常低，只有0.3左右      
1）bottleneck特征提取，效果不好，最后没有采用        
2）densenet xception模型并联融合，精度与单个模型精度相当，但是在模型存贮过程中会超出内存限制，因此也没有采用    


In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


['speech-train2', 'model3-20epoch081', 'speech-test', 'model3-1cnn2', 'model3-10epoch058', 'multimodel3', 'multimodel2', 'model3-20epoch059', 'model3-30epoch0398', 'densemodel', 'model3', 'multimodel']


In [2]:
# 查看图像信息
import matplotlib.image as mpimg
im = mpimg.imread('../input/speech-test/test/test/23330.png')
print(im.shape)


(128, 173, 4)


## 1. 读取数据

In [4]:

from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image 

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,AveragePooling2D,Convolution2D,Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
import keras.backend as K
import numpy as np
import pandas as pd

Using TensorFlow backend.


##  2-1.建立CNN自定义模型 

In [5]:

# def CNN_model(train_generator, valid_generator, epochs=50, batch_size=128):  # 生成器generator格式
def CNN_model(train_generator, valid_generator, epochs=50, batch_size=32):
    # 模型框架构建
    model = Sequential()

#     model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
#                      activation='relu', input_shape=(128, 173, 3)))
#     model.add(BatchNormalization())
    
#     model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
#                      activation='relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPool2D(pool_size=(2, 2)))
#     model.add(Dropout(.25))
    
#     model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
#                      activation='relu'))
#     model.add(BatchNormalization())
    
#     model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
#                      activation='relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
#     model.add(Dropout(.25))
    
#     model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same',
#                      activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(.25))
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(.25))
#     model.add(Dense(3, activation='softmax'))   # 3个节点

    model.add(BatchNormalization(input_shape=(64, 87, 3))) #
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu',))  #(128, 173, 3)
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(.2))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(.25))
    
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(.25))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Dropout(.3))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(3, activation='softmax'))   # 3个节点
    

    # 模型编译
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])#

    # 学习率的annealing模拟退火
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                patience=4,
                                                verbose=2,
                                                factor=.5,
                                                min_lr=.00001)

    # 图像随机旋转10度, 0.9-1.1随机缩放，水平和垂直方向随机便宜0.1幅度
#     rotation_range = 10
#     zoom_range = .1
#     height_shift_range = .1
#     width_shift_range = .1
#     # 数据扩增
#     data_gen = ImageDataGenerator(rotation_range=rotation_range,
#                                   zoom_range=zoom_range,
#                                   height_shift_range=height_shift_range,
#                                   width_shift_range=width_shift_range)
#     data_gen.fit(x_train)

    # model.fit_generator会返回一个history对象，该对象记录了训练过程中的运行输出
    # data_gen.flow是一个keras的生成器函数，会无限地进行循环，不会返回或退出
    history = model.fit_generator(train_generator,
                                  epochs=epochs,
                                  validation_data=valid_generator,
                                  validation_steps=val_shape // batch_size,
                                  verbose=2,
                                  steps_per_epoch=train_shape // batch_size,
                                  callbacks=[learning_rate_reduction]
                                 )

    return history, model



In [6]:
# 单次验证                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# history, model = CNN_model(train_generator, valid_generator)
# i = 0 
# print('fold {0:d}: Epochs=50, Training accuracy={1:.5f}, Validation accuracy={2:.5f}'.format(i + 1,
#                                                                             max(history.history['acc']),
#                                                                             max(history.history['val_acc'])))

In [7]:
# 单次预测
# pred = model.predict_generator(test_generator,steps= len(test_generator))

In [8]:

# df = pd.DataFrame(columns=['file_id','accent'])  #输出文件
# i= 0

# predicted_class_indices = np.argmax(pred, axis=1)
# labels = (train_generator.class_indices)
# label = dict((v,k) for k,v in labels.items())
# print('labels',labels)
# print('label',label)
# # 建立代码标签与真实标签的关系
# predictions = [label[i] for i in predicted_class_indices]

# # #建立预测结果和文件名之间的关系
# filenames = test_generator.filenames
# for idx in range(len(filenames )):
#     df.at[i,'file_id'] = filenames[idx]
#     df.at[i,'accent'] = (int(predictions[idx]))
#     i += 1
# #     print('predict  %d' % (int(predictions[idx])))
# #     print('title    %s' % filenames[idx])
# df['file_id'] = df['file_id'].apply(lambda x: x.split('/')[-1].split('.')[0])
# df = df.sort_values(by = 'file_id' , ascending = True)
# df.head()


In [9]:
# df.accent.value_counts()

In [10]:
# 输出结果

# df.to_csv('submission.csv',index = False)
# print(os.listdir("../input"))

## 2-2. DenseNet

### 2-2-0 模型概览

In [11]:
from keras.applications.densenet import DenseNet201,preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
 

### 2-2-1 训练模型

In [12]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import keras
from keras import __version__
from keras.applications.densenet import DenseNet201,preprocess_input
 
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, LearningRateScheduler
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image 


from keras.models import Sequential
import keras.backend as K
import numpy as np
import pandas as pd

In [13]:
train_df = pd.read_csv('../input/speech-train2/index.csv')
train_df['accent'] = train_df['accent'].astype('str')
print(type(train_df['accent'].values))
train_df['file_id'] = "../input/speech-train2/train/train/" +train_df['file_id']
train_df.head()

<class 'numpy.ndarray'>


,file_id,accent
0,../input/speech-train2/train/train/10000.png,1
1,../input/speech-train2/train/train/10001.png,1
2,../input/speech-train2/train/train/10002.png,0
3,../input/speech-train2/train/train/10003.png,2
4,../input/speech-train2/train/train/10004.png,1


In [14]:
#  训练集、测试集
# 数据准备
IM_WIDTH, IM_HEIGHT = 128, 173 #densenet指定的图片尺寸

nb_classes = 3 # 分类数
batch_size = int(32)        
target_size = (128,173)
train_shape = 3600  # 训练样本个数
val_shape = 900  # 验证样本个数


# 训练数据与测试数据

datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,   #None
                             rescale=1./255.,
                            zca_whitening = True,
                             validation_split=0.2)
 
train_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None, 
                        x_col="file_id", 
                        y_col="accent",
                        has_ext=False,
                        subset="training",
                        batch_size=batch_size,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=target_size)  #(128, 173)


valid_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None,
                        x_col="file_id",
                        y_col="accent",
                        has_ext=False,
                        subset="validation",
                        batch_size=batch_size, #1
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=target_size)




/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 3600 validated image filenames belonging to 3 classes.
Found 900 validated image filenames belonging to 3 classes.


### (1) DenseNet Model

In [15]:
'''
# 添加新层
def add_new_last_layer(base_model, nb_classes):

   # 添加最后的层
   # 输入
   # base_model和分类数量
   # 输出
   # 新的keras的model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    return model
 

#搭建模型
model = DenseNet201(include_top=False)  #  可尝试是否制定 预训练好的权重
model = add_new_last_layer(model, nb_classes)
#model.load_weights('../model/checkpoint-02e-val_acc_0.82.hdf5')  第二次训练可以接着第一次训练得到的模型接着训练
model.compile(optimizer = 'adam',
#               optimizer=SGD(lr=0.001, momentum=0.9,decay=0.0001,nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])
 


#更好地保存模型 Save the model after every epoch.
output_model_file = 'densenet_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'
# keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
checkpoint = ModelCheckpoint(output_model_file, monitor='val_acc',
                             verbose=1, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                patience=4,
                                                verbose=2,
                                                factor=.3,
                                                min_lr=.00001)
 
#开始训练
history_ft = model.fit_generator(
                                train_generator,
                                nb_epoch=30,
                                validation_data=valid_generator,
                                validation_steps=val_shape // batch_size,
#                                 verbose=2,
                                steps_per_epoch=train_shape // batch_size,
                                callbacks=[learning_rate_reduction,checkpoint],
)
'''
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r-')
    plt.plot(epochs, val_acc, 'b')
    plt.legend(['training','val'])
    plt.title('Training and validation accuracy')
    
    plt.figure()
    plt.plot(epochs, loss, 'r-')
    plt.plot(epochs, val_loss, 'b-')
    plt.title('Training and validation loss')
    plt.legend(['training','val'])
    plt.show()

    
# 训练的acc_loss图
# plot_training(history_ft)


### (2) Xception Model

In [16]:
from keras.applications.xception import Xception,preprocess_input
from keras.layers import Input,Dense,Dropout

In [17]:
'''
nb_classes = 3 # 分类数
nb_epoch = int(30)                # epoch数量
batch_size = int(32)        
target_size = (128,173)
train_shape = 3600  # 训练样本个数
val_shape = 900  # 验证样本个数


base_model = Xception(include_top=False ) #input_tensor=input_image,  
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predicts =Dense(3, activation='softmax')(x) # for i in range(3) ] 

                                          
xception_model = Model(inputs=base_model.input, outputs=predicts)  #
xception_model.compile(optimizer='adam',
                       loss='categorical_crossentropy', metrics=['accuracy'])


x_output_model_file = 'x_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'

x_checkpoint = ModelCheckpoint(x_output_model_file, monitor='val_acc',
                             verbose=1, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                patience=4,
                                                verbose=2,
                                                factor=.3,
                                                min_lr=.00001)

xception_history_ft = xception_model.fit_generator(
                                train_generator,
                                nb_epoch=nb_epoch,  #
                                validation_data=valid_generator,
                                validation_steps=val_shape // batch_size,
#                                 verbose=2,
                                steps_per_epoch=train_shape // batch_size,
                                callbacks=[learning_rate_reduction,x_checkpoint], #
)
 
    
# 训练的acc_loss图
plot_training(xception_history_ft)
'''


"\nnb_classes = 3 # 分类数\nnb_epoch = int(30)                # epoch数量\nbatch_size = int(32)        \ntarget_size = (128,173)\ntrain_shape = 3600  # 训练样本个数\nval_shape = 900  # 验证样本个数\n\n\nbase_model = Xception(include_top=False ) #input_tensor=input_image,  \nx = base_model.output\nx = GlobalAveragePooling2D()(x)\n# x = Dropout(0.5)(x)\n# x = Dense(1024, activation='relu')(x)\nx = Dropout(0.5)(x)\npredicts =Dense(3, activation='softmax')(x) # for i in range(3) ] \n\n                                          \nxception_model = Model(inputs=base_model.input, outputs=predicts)  #\nxception_model.compile(optimizer='adam',\n                       loss='categorical_crossentropy', metrics=['accuracy'])\n\n\nx_output_model_file = 'x_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'\n\nx_checkpoint = ModelCheckpoint(x_output_model_file, monitor='val_acc',\n                             verbose=1, save_best_only=True)\nlearning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',\n              

## (3) multi model测试


## 存储 Densenet Xception 模型


In [18]:
# 存储 

import os
files = os.listdir("../working")
print(files)
x_model_checkpoint = []
dense_model_checkpoint = []
for file in files:
    if file.startswith('densenet_checkpoint'):  # densenet checkpoint
        dense_model_checkpoint.append(file)
    if file.startswith('x_checkpoint'):  # xception checkpoint
        x_model_checkpoint.append(file)
        
x_model_checkpoint = sorted(x_model_checkpoint)
dense_model_checkpoint = sorted(dense_model_checkpoint)
print()
print(dense_model_checkpoint)
print()
print(x_model_checkpoint)


['__output__.json', '__notebook__.ipynb']

[]

[]


## 加载 Densenet Xception 模型

In [19]:
import h5py
import keras

In [20]:
# xception=Xception(include_top=False,weights=None,) #input_shape=(224,224,3)
# dense_model = load_model('densenet_checkpoint-30e-val_acc_0.77.hdf5')
# # dense_model.save_weights('densenet_30e_0.77.h5')
# xception_model = load_model('x_checkpoint-22e-val_acc_0.82.hdf5')
# xception_model.save_weights('xception_22e_0.82.h5')

## 直接输出的话disk爆掉
# dense_model = load_model(dense_model_checkpoint[-1])
# xception_model = load_model(x_model_checkpoint[-1])

dense_model = load_model('../input/multimodel/densenet_checkpoint-26e-val_acc_0.80.hdf5')
xception_model = load_model('../input/multimodel2/x_checkpoint-26e-val_acc_0.80.hdf5')
# print(x_model_checkpoint[-1])
# xception_model = load_model(x_model_checkpoint[-1])



In [21]:
import os
print(os.listdir('../working'))

['__output__.json', '__notebook__.ipynb']


## DenseNet Xception 模型融合

In [22]:
def Multimodel(dense_model,xception_model,all_weights_path=None,class_num=3,cnn_no_vary=False):
    '''
    获取densent201,xinception并联的网络
    此处的cnn_weights_path是个列表是densenet和xception的卷积部分的权值
    '''
    input_layer=Input(shape=(128,173,3))
    dense=DenseNet201(include_top=False,weights=None) #,input_shape=(224,224,3)
    xception=Xception(include_top=False,weights=None,) #input_shape=(224,224,3)
    
    xception.load_weights('../input/multimodel2/x_checkpoint-26e-val_acc_0.80.hdf5', by_name = True)
    dense.load_weights('../input/multimodel/densenet_checkpoint-26e-val_acc_0.80.hdf5', by_name = True)
 
 
    if cnn_no_vary:
        for i,layer in  enumerate(dense.layers):
            dense.layers[i].trainable=False
        for i,layer in enumerate(xception.layers):
            xception.layers[i].trainable=False
       
 
   # if cnn_weights_path!=None:
#         dense.load_weights(cnn_weights_path[0])
#         xception.load_weights(cnn_weights_path[1])
##     dense = dense_model
##     xception = xception_model
#     dense_model.name = 'model_dense'   # m
#     xception_model.name = 'model_xception'
    
#     dense=dense_model(input_layer)
#     xception=xception_model(input_layer)
    dense=dense (input_layer)
    xception=xception (input_layer)
    #对dense_121和xception进行全局最大池化
    top1_model=GlobalMaxPooling2D()(dense)# dense #data_format='channels_last'GlobalMaxPooling2D
    top2_model=GlobalMaxPooling2D()(xception)#xception#data_format='channels_last'GlobalMaxPooling2D


    print(top1_model.shape,top2_model.shape)
    #把top1_model和top2_model连接起来
    t=keras.layers.Concatenate(axis=1)([top1_model,top2_model])
    #第一个全连接层
    top_model=Dense(units=128,activation="relu")(t)
    top_model=Dropout(rate=0.5)(top_model)
#     top_model=Dense(units=256,activation="relu")(t)
#     top_model=Dropout(rate=0.7)(top_model)
    top_model=Dense(units=class_num,activation="softmax")(top_model)

#     
    model=Model(inputs=input_layer,outputs=top_model)

 
    #加载全部的参数
    if all_weights_path:
        model.load_weights(all_weights_path)
    return model

In [23]:

multi_model=Multimodel(dense_model,xception_model,class_num=3)
# multi_model = load_model('../input/multimodel3/multi_checkpoint-23e-val_acc_0.82.hdf5',compile=False)

(?, 1920) (?, 2048)


In [24]:
# for layer in multi_model.layers:
#     layer.trainable = False#原来的不训练
        
# inp=multi_model.input
# x=multi_model.output
# x = Input(( 128, 173, 3))
# # l=Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
# #                      activation='relu', input_shape=(None,128, 173, 3))(x)
# # # Conv2D(64, 3, 3, border_mode='same',init='glorot_uniform')(reshape)
# # # l=PReLU()(l)
# # l=BatchNormalization()(l)

# l=GlobalAveragePooling2D()(x)
# den=Dense(512,name="fine_dense4")(x)
# l=PReLU()(den)
# # l=GlobalAveragePooling2D()(l)
# l=Dropout(0.5)(l)
# # result=Dense(3,activation="softmax")(l)
# den=Dense(1024,name="fine_dense6")(x)
# l=PReLU()(den)

# l=Dropout(0.7)(l)
# # l=GlobalAveragePooling2D()(l)
# result=Dense(3,activation="softmax")(l)


# multi_model=Model(input=inp,outputs=result)

#编译model



# 模型编译
multi_model.compile(optimizer='adam',
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])

multi_output_model_file = 'multi_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'

multi_checkpoint = ModelCheckpoint(multi_output_model_file, monitor='val_acc',
                             verbose=1, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                patience=4,
                                                verbose=2,
                                                factor=.3,
                                                min_lr=.00001)

multi_history_ft = multi_model.fit_generator(
                                train_generator,
                                nb_epoch=30,  #
                                validation_data=valid_generator,
                                validation_steps=val_shape // batch_size,

                                steps_per_epoch=train_shape // batch_size,
                                callbacks=[learning_rate_reduction,multi_checkpoint], #
)
 


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., validation_steps=28, steps_per_epoch=112, callbacks=[<keras.ca..., epochs=30)`


Epoch 1/30


/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


112/112 [==============================] - 154s 1s/step - loss: 0.5206 - acc: 0.9121 - val_loss: 3.5892 - val_acc: 0.6440

Epoch 00001: val_acc improved from -inf to 0.64397, saving model to multi_checkpoint-01e-val_acc_0.64.hdf5
Epoch 2/30
112/112 [==============================] - 63s 562ms/step - loss: 0.1481 - acc: 0.9587 - val_loss: 1.8858 - val_acc: 0.7373

Epoch 00002: val_acc improved from 0.64397 to 0.73733, saving model to multi_checkpoint-02e-val_acc_0.74.hdf5
Epoch 3/30
112/112 [==============================] - 61s 549ms/step - loss: 0.1052 - acc: 0.9660 - val_loss: 2.5077 - val_acc: 0.6601

Epoch 00003: val_acc did not improve from 0.73733
Epoch 4/30
112/112 [==============================] - 61s 545ms/step - loss: 0.0884 - acc: 0.9752 - val_loss: 1.8526 - val_acc: 0.7442

Epoch 00004: val_acc improved from 0.73733 to 0.74424, saving model to multi_checkpoint-04e-val_acc_0.74.hdf5
Epoch 5/30
112/112 [==============================] - 62s 551ms/step - loss: 0.0781 - acc: 0

## 存储融合模型

In [25]:

files = os.listdir("../working")
print(files)
print()
multi_checkpoint = []
for file in files:

    if file.startswith('multi_checkpoint'):  # multi checkpoint
        multi_checkpoint.append(file)
multi_checkpoint = sorted(multi_checkpoint)      
print((multi_checkpoint))

['multi_checkpoint-15e-val_acc_0.80.hdf5', 'multi_checkpoint-02e-val_acc_0.74.hdf5', 'multi_checkpoint-04e-val_acc_0.74.hdf5', 'multi_checkpoint-11e-val_acc_0.78.hdf5', 'multi_checkpoint-12e-val_acc_0.79.hdf5', 'multi_checkpoint-26e-val_acc_0.81.hdf5', 'multi_checkpoint-01e-val_acc_0.64.hdf5', '__output__.json', 'multi_checkpoint-10e-val_acc_0.77.hdf5', '__notebook__.ipynb', 'multi_checkpoint-09e-val_acc_0.76.hdf5']

['multi_checkpoint-01e-val_acc_0.64.hdf5', 'multi_checkpoint-02e-val_acc_0.74.hdf5', 'multi_checkpoint-04e-val_acc_0.74.hdf5', 'multi_checkpoint-09e-val_acc_0.76.hdf5', 'multi_checkpoint-10e-val_acc_0.77.hdf5', 'multi_checkpoint-11e-val_acc_0.78.hdf5', 'multi_checkpoint-12e-val_acc_0.79.hdf5', 'multi_checkpoint-15e-val_acc_0.80.hdf5', 'multi_checkpoint-26e-val_acc_0.81.hdf5']


### BottleNeck 特征提升准确率

In [26]:

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16

In [27]:
'''
######### 1. 生成训练集与测试集的bottleneck特征

# include_top：是否保留顶层的3个全连接网络
# 'imagenet'代表加载预训练权重
model = VGG16(include_top=False, weights='imagenet')
# 加载pre-model的权重
model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')


# 训练集图像生成器,以文件夹路径为参数,生成经过数据提升/归一化后的数据


# 得到bottleneck feature
# 使用一个生成器作为数据源预测模型，生成器应返回与test_on_batch的输入数据相同类型的数据
bottleneck_features_train = model.predict_generator(train_generator, steps=len(train_generator)) 
print (bottleneck_features_train) 
# steps是生成器要返回数据的轮数
# 将得到的特征记录在numpy array里
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)
bottleneck_features_validation = model.predict_generator(val_generator, steps=len(val_generator)) 
# 一个epoch有800张图片,验证集
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
'''

"\n######### 1. 生成训练集与测试集的bottleneck特征\n\n# include_top：是否保留顶层的3个全连接网络\n# 'imagenet'代表加载预训练权重\nmodel = VGG16(include_top=False, weights='imagenet')\n# 加载pre-model的权重\nmodel.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')\n\n\n# 训练集图像生成器,以文件夹路径为参数,生成经过数据提升/归一化后的数据\n\n\n# 得到bottleneck feature\n# 使用一个生成器作为数据源预测模型，生成器应返回与test_on_batch的输入数据相同类型的数据\nbottleneck_features_train = model.predict_generator(train_generator, steps=len(train_generator)) \nprint (bottleneck_features_train) \n# steps是生成器要返回数据的轮数\n# 将得到的特征记录在numpy array里\nnp.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)\nbottleneck_features_validation = model.predict_generator(val_generator, steps=len(val_generator)) \n# 一个epoch有800张图片,验证集\nnp.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)\n"

 ### (3) InceptionResNetV2 Model

In [28]:
from keras.applications.nasnet import NASNetLarge
from keras.applications.inception_resnet_v2 import InceptionResNetV2

Mean acc  : - training: 0.7057491324232396           
Mean acc  : - val: 0.5350085034013604         
Mean loss : - training: 0.7226675915719312            
Mean loss : - val: 2.2375190057559884'
准确率并不高

## Xception Densenet 迁移学习

In [29]:
# train_df = pd.read_csv('../input/speech-train2/index.csv')
# train_df['accent'] = train_df['accent'].astype('str')
# print(type(train_df['accent'].values))
# train_df['file_id'] = "../input/speech-train2/train/train/" +train_df['file_id']
# train_df.head()

In [30]:
'''
#  训练集、测试集
# 数据准备
IM_WIDTH, IM_HEIGHT = 128, 173 #densenet指定的图片尺寸

nb_classes = 3 # 分类数
batch_size = int(32)        
target_size = (128,173)



# 训练数据与测试数据
datagen = image.ImageDataGenerator(preprocessing_function=None,   #preprocess_input
                             rescale=1./255.,
                            zca_whitening = True,    
                             validation_split=0.2)
 
train_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory=None, 
                        x_col="file_id", 
                        y_col="accent",
                        has_ext=False,
                        batch_size=batch_size,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=target_size)  #(128, 173)


folder_dir = "../input/speech-test/test/test/"  #测试集

test_df = pd.DataFrame(columns=['file_id','accent'])
i= 0 

for filename in os.listdir(folder_dir):
    file = folder_dir+filename
    test_df.at[i,'file_id'] = file
    i+=1

print(test_df.shape)

test_datagen=image.ImageDataGenerator(rescale=1./255.,
                                     preprocessing_function=None,   #preprocess_input
                             )

test_generator=test_datagen.flow_from_dataframe(
                            dataframe=test_df,
                            directory=None,
                            x_col="file_id",
                            y_col=None,
                            has_ext=False,
                            batch_size=batch_size,#1
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=target_size)
test_df.head()
'''

'\n#  训练集、测试集\n# 数据准备\nIM_WIDTH, IM_HEIGHT = 128, 173 #densenet指定的图片尺寸\n\nnb_classes = 3 # 分类数\nbatch_size = int(32)        \ntarget_size = (128,173)\n\n\n\n# 训练数据与测试数据\ndatagen = image.ImageDataGenerator(preprocessing_function=None,   #preprocess_input\n                             rescale=1./255.,\n                            zca_whitening = True,    \n                             validation_split=0.2)\n \ntrain_generator=datagen.flow_from_dataframe(\n                        dataframe=train_df,\n                        directory=None, \n                        x_col="file_id", \n                        y_col="accent",\n                        has_ext=False,\n                        batch_size=batch_size,\n                        seed=42,\n                        shuffle=True,\n                        class_mode="categorical",\n                        target_size=target_size)  #(128, 173)\n\n\nfolder_dir = "../input/speech-test/test/test/"  #测试集\n\ntest_df = pd.DataFrame(columns=[\'fi

In [31]:
'''
## 1. 生成迁移学习特征变量
def write_gap(MODEL, model_name,lambda_func=None):
    width = 173
    height = 128
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x,  weights='imagenet',include_top=False, )  #weights='imagenet',pooling='avg'
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(0.5)(x)
#     predicts =Dense(3, activation='softmax')(x) # for i in range(3) ] 

#     model = Model(base_model.input, predicts)
    model = Model(base_model.input,GlobalAveragePooling2D()(base_model.output) )
    
    train = model.predict_generator(train_generator, len(train_generator))
    test = model.predict_generator(test_generator, len(test_generator))

    with h5py.File("gap_%s.h5"%model_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        '''

'\n## 1. 生成迁移学习特征变量\ndef write_gap(MODEL, model_name,lambda_func=None):\n    width = 173\n    height = 128\n    input_tensor = Input((height, width, 3))\n    x = input_tensor\n    if lambda_func:\n        x = Lambda(lambda_func)(x)\n    base_model = MODEL(input_tensor=x,  weights=\'imagenet\',include_top=False, )  #weights=\'imagenet\',pooling=\'avg\'\n#     x = base_model.output\n#     x = GlobalAveragePooling2D()(x)\n#     x = Dropout(0.5)(x)\n#     predicts =Dense(3, activation=\'softmax\')(x) # for i in range(3) ] \n\n#     model = Model(base_model.input, predicts)\n    model = Model(base_model.input,GlobalAveragePooling2D()(base_model.output) )\n    \n    train = model.predict_generator(train_generator, len(train_generator))\n    test = model.predict_generator(test_generator, len(test_generator))\n\n    with h5py.File("gap_%s.h5"%model_name) as h:\n        h.create_dataset("train", data=train)\n        h.create_dataset("test", data=test)\n        h.create_dataset("label", data=tra

In [32]:
'''
write_gap(Xception,'Xception')
write_gap(DenseNet201,'DenseNet201')
'''

"\nwrite_gap(Xception,'Xception')\nwrite_gap(DenseNet201,'DenseNet201')\n"

In [33]:
## 2. 载入特征变量
# np.random.seed(2019)

# X_train = []
# X_test = []

# for filename in ['gap_DenseNet201.h5', 'gap_Xception.h5',]:
#     with h5py.File(filename, 'r') as h:
        
#         X_train.append(np.array(h['train']))
#         X_test.append(np.array(h['test']))
#         y_train = np.array(h['label'])
#         print(h['train'],h['test'])

# X_train = np.concatenate(X_train, axis=1)
# X_test = np.concatenate(X_test, axis=1)
# y_train = keras.utils.to_categorical(y_train , num_classes=3)



In [34]:
## 3. 构建模型
# print(X_train.shape)
# input_tensor = Input(X_train.shape[1:])
# x = input_tensor
# # x = Dropout(0.5)(x)
# # x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(3, activation='softmax')(x)
# model = Model( input_tensor,x) #

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy', #categorical
#               metrics=['accuracy'])

In [35]:
## 4. 训练模型


# model_history = model.fit(X_train, 
#                     y_train,
#                     batch_size=32,
#                     epochs=20,
#                     verbose=1,
#                     validation_split=0.2,
# #                     callbacks = [keras.callbacks.TensorBoard(log_dir='./Graph',)] #TensorBoard(log_dir='./Graph')
# )
# model.save('model.h5')

## 测试数据集

In [36]:
# 测试集

folder_dir = "../input/speech-test/test/test/"  #测试集

test_df = pd.DataFrame(columns=['file_id','accent'])
i= 0 

for filename in os.listdir(folder_dir):
    file = folder_dir+filename
    test_df.at[i,'file_id'] = file
    i+=1

print(test_df.shape)

test_datagen=image.ImageDataGenerator(rescale=1./255.,
                                     preprocessing_function=preprocess_input,   #None
                             )

test_generator=test_datagen.flow_from_dataframe(
                            dataframe=test_df,
                            directory=None,
                            x_col="file_id",
                            y_col=None,
                            has_ext=False,
                            batch_size=batch_size,#1
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=target_size)
test_df.head()

(5377, 2)
Found 5377 validated image filenames.


,file_id,accent
0,../input/speech-test/test/test/21352.png,NaN
1,../input/speech-test/test/test/21121.png,NaN
2,../input/speech-test/test/test/23559.png,NaN
3,../input/speech-test/test/test/20365.png,NaN
4,../input/speech-test/test/test/24472.png,NaN


## 模型预测测试集 

In [37]:
def output(model , test_generator, name):
    # 模型预测
    
    pred = model.predict_generator(test_generator,steps= len(test_generator)) 

    
    df = pd.DataFrame(columns=['file_id','accent'])  #输出文件
    i= 0

    predicted_class_indices = np.argmax(pred, axis=1)

    
    # # #建立预测结果和文件名之间的关系
    filenames = test_generator.filenames
    for idx in range(len(filenames )):
        df.at[i,'file_id'] = filenames[idx]
        df.at[i,'accent'] = (int(predicted_class_indices[idx]))
        i += 1
#         print('predict  %d' % (int(predicted_class_indices[idx])),'title    %s' % filenames[idx])


    df['file_id'] = df['file_id'].apply(lambda x: x.split('/')[-1].split('.')[0])
    df = df.sort_values(by = 'file_id' , ascending = True)
    print(df.head())
    df.to_csv(name,index = False)
    return df


In [38]:
### 加载已训练好的模型测试  模型过拟合？？？？？？？？？？？
'''
multi_model = load_model('../input/multimodel3/multi_checkpoint-23e-val_acc_0.82.hdf5',compile=False)



multi_model.compile(optimizer='adam',
                       loss='categorical_crossentropy', metrics=['accuracy'])


# x_output_model_file = 'loadmulti_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'

# x_checkpoint = ModelCheckpoint(x_output_model_file, monitor='val_acc',
#                              verbose=1, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                                patience=4,
                                                verbose=2,
                                                factor=.3,
                                                min_lr=.00001)

multi_history_ft = multi_model.fit_generator(
                                train_generator,
                                nb_epoch=30,  #
                                validation_data=valid_generator,
                                validation_steps=val_shape // batch_size,
                                steps_per_epoch=train_shape // batch_size,
                                callbacks=[learning_rate_reduction,], #x_checkpoint
)
'''



"\nmulti_model = load_model('../input/multimodel3/multi_checkpoint-23e-val_acc_0.82.hdf5',compile=False)\n\n\n\nmulti_model.compile(optimizer='adam',\n                       loss='categorical_crossentropy', metrics=['accuracy'])\n\n\n# x_output_model_file = 'loadmulti_checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'\n\n# x_checkpoint = ModelCheckpoint(x_output_model_file, monitor='val_acc',\n#                              verbose=1, save_best_only=True)\nlearning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',\n                                                patience=4,\n                                                verbose=2,\n                                                factor=.3,\n                                                min_lr=.00001)\n\nmulti_history_ft = multi_model.fit_generator(\n                                train_generator,\n                                nb_epoch=30,  #\n                                validation_data=valid_generator,\n               

In [39]:
files = os.listdir("../working")
print(files)
print()
loadmulti_checkpoint = []
for file in files:

    if file.startswith('loadmulti_checkpoint'):  # multi checkpoint
        loadmulti_checkpoint.append(file)
loadmulti_checkpoint = sorted(multi_checkpoint)      
print((loadmulti_checkpoint))

['multi_checkpoint-15e-val_acc_0.80.hdf5', 'multi_checkpoint-02e-val_acc_0.74.hdf5', 'multi_checkpoint-04e-val_acc_0.74.hdf5', 'multi_checkpoint-11e-val_acc_0.78.hdf5', 'multi_checkpoint-12e-val_acc_0.79.hdf5', 'multi_checkpoint-26e-val_acc_0.81.hdf5', 'multi_checkpoint-01e-val_acc_0.64.hdf5', '__output__.json', 'multi_checkpoint-10e-val_acc_0.77.hdf5', '__notebook__.ipynb', 'multi_checkpoint-09e-val_acc_0.76.hdf5']

['multi_checkpoint-01e-val_acc_0.64.hdf5', 'multi_checkpoint-02e-val_acc_0.74.hdf5', 'multi_checkpoint-04e-val_acc_0.74.hdf5', 'multi_checkpoint-09e-val_acc_0.76.hdf5', 'multi_checkpoint-10e-val_acc_0.77.hdf5', 'multi_checkpoint-11e-val_acc_0.78.hdf5', 'multi_checkpoint-12e-val_acc_0.79.hdf5', 'multi_checkpoint-15e-val_acc_0.80.hdf5', 'multi_checkpoint-26e-val_acc_0.81.hdf5']


In [40]:
'''
loadmulti_model = load_model(loadmulti_checkpoint[-1],compile=False)
df = output(loadmulti_model ,test_generator,'load_multimodel_submission.csv')
print(df.accent.value_counts())
'''

"\nloadmulti_model = load_model(loadmulti_checkpoint[-1],compile=False)\ndf = output(loadmulti_model ,test_generator,'load_multimodel_submission.csv')\nprint(df.accent.value_counts())\n"

In [41]:

# densenet预测输出
# dense_model = load_model(dense_model_checkpoint[-1]) #选择最高的一个模型  #checkpoint[-1]
# df = output(dense_model ,test_generator,'densenet_submission.csv')
# print(df.accent.value_counts())

# xception模型预测输出
# xception_model = load_model(x_model_checkpoint[-1]) 
df = output(xception_model ,test_generator,'xception_submission.csv')  # 利用已加载的
print(df.accent.value_counts())

# 融合模型预测输出
multi_model = load_model(multi_checkpoint[-1],compile=False) 
df = output( multi_model,test_generator,'multi_submission.csv')
print(df.accent.value_counts())


     file_id accent
1753   20000      1
2831   20001      1
2603   20002      1
1355   20003      1
3850   20004      1
1    5377
Name: accent, dtype: int64
     file_id accent
1753   20000      2
2831   20001      0
2603   20002      2
1355   20003      0
3850   20004      1
2    1998
0    1905
1    1474
Name: accent, dtype: int64


In [42]:
# df.to_csv('densenet_submission.csv',index = False)
# df.to_csv('xception_submission.csv',index = False)

已建好模型测试

In [43]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# from PIL import Image
# import tensorflow as tf
# from keras.preprocessing.image import img_to_array,load_img
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input
# import keras
# from sklearn.metrics import accuracy_score
# print(keras.__version__)

# model = tf.keras.models.load_model('../input/model3-30epoch0398/speech_6_2.h5' , compile = False)# 3个类别0.40



In [44]:
# folder_dir = "../input/speech-test/test/test/"  #测试集

# test_df = pd.DataFrame(columns=['file_id','accent'])
# i= 0 

# for filename in os.listdir(folder_dir):
#     file = folder_dir+filename
#     test_df.at[i,'file_id'] = file
#     i+=1

# print(test_df.shape)

# test_datagen=image.ImageDataGenerator(rescale=1./255.,
#                                      preprocessing_function=None,   #preprocess_input
                                   
#                              )

# test_generator=test_datagen.flow_from_dataframe(
#                             dataframe=test_df,
#                             directory=None,
#                             x_col="file_id",
#                             y_col=None,
#                             has_ext=False,
#                             batch_size=32,#1
#                             seed=42,
#                             shuffle=False,
#                             class_mode=None,
#                             target_size=(128,173))


In [45]:


# # pred = model.predict_generator(test_generator,steps= len(test_generator))
# df = pd.DataFrame(columns=['file_id','accent'])  #输出文件
# i= 0

# predicted_class_indices = np.argmax(pred, axis=1)

# # # #建立预测结果和文件名之间的关系
# filenames = test_generator.filenames
# for idx in range(len(filenames )):
#     df.at[i,'file_id'] = filenames[idx]
#     df.at[i,'accent'] = (int(predictions[idx]))
#     i += 1
#     print('predict  %d' % (int(predictions[idx])),'title    %s' % filenames[idx])

    
# df['file_id'] = df['file_id'].apply(lambda x: x.split('/')[-1].split('.')[0])
# df = df.sort_values(by = 'file_id' , ascending = True)
# df.head()


In [46]:
# df.accent.value_counts()
# df.to_csv('loadmodel_submission.csv',index = False)